# Sandbox

Let's play!

In [ ]:
import rl_pulse.spin_simulation as ss
import numpy as np
from scipy import linalg

In [ ]:
N = 4
dim = 2**N
coupling = 5e3
delta = 500
pulse_width = 1e-6
delay = 10e-6

(x,y,z) = (ss.x, ss.y, ss.z)
(X,Y,Z) = ss.get_total_spin(N, dim)

Hdip, Hint = ss.get_H(N, dim, coupling, delta)
H_target = ss.get_H_WHH_0(X, Y, Z, delta)

In [ ]:
Utarget_step = ss.get_propagator(H_target, (pulse_width + delay))
        
# define actions
Udelay = linalg.expm(-1j * Hint * delay)
Ux = linalg.expm(-1j * (X*np.pi/2 + Hint*pulse_width))
Uxbar = linalg.expm(-1j * (X*-np.pi/2 + Hint*pulse_width))
Uy = linalg.expm(-1j * (Y*np.pi/2 + Hint*pulse_width))
Uybar = linalg.expm(-1j * (Y*-np.pi/2 + Hint*pulse_width))
Ux = Udelay @ Ux
Uxbar = Udelay @ Uxbar
Uy = Udelay @ Uy
Uybar = Udelay @ Uybar
Udelay = linalg.expm(-1j*(Hint*(pulse_width + delay)))

In [ ]:
U1 = Uybar @ Udelay @ Uy @ Ux @ Udelay @ Uxbar
U2 = Uxbar @ Uybar @ Udelay @ Uy @ Ux @ Udelay
U3 = Udelay @ Uxbar @ Uybar @ Udelay @ Uy @ Ux

In [ ]:
ss.fidelity(U2, U3)

In [ ]:
ss.metric1(U2, U3)

In [ ]:
UWHH = ss.get_U_WHH(Hint, delay, pulse_width, X, Y)

In [ ]:
ss.fidelity(U2, UWHH)

OK, after confusing myself for forever with rotations, I think I have it figured out (and I think there's a typo in table 4-2 in _Haeberlen_, p58). a $P_x$ rotation (90 degree rotation about the x axis) means rotating vectors _clockwise_ by 90 degrees when you look at the origin from the x axis. Then a $P_\overline{x}$ rotation rotates vectors _counterclockwise_, which puts vectors along y to vectors along z. $P_\overline{x} = e^{i \pi/2 X}$.

$$
e^{-i \pi/2 X} Z e^{i \pi/2 X} = Y
$$

Similarly, $P_y$ rotates vectors _clockwise_ when looking at the origin from the y axis. So vectors along x are rotated to along z.

$$
e^{i \pi/2 Y} Z e^{-i \pi/2 Y} = X
$$

$$
e^{-i \pi/2 X} X e^{i \pi/2 X} = X \text{ (commutes with itself.)}
$$

And $P_z$ rotates vectors _clockwise_ when looking at the origin from the z axis. So vectors along y are rotated to along x.

How confusing...

In [ ]:
np.round(linalg.expm(-1j * x * np.pi/2) @ z @
         linalg.expm(+1j * x * np.pi/2), decimals = 3) - y